<a href="https://colab.research.google.com/github/divyanshgupt/Unreliable-Transmission/blob/main/Superspike_Multilayer_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Dependencies
import numpy as np
from matplotlitb import pyplot as plt

import torch
from tqdm import tqdm

In [ ]:
dtype = torch.float
device = torch.device('cpu')

# device = torch.device('cuda:0')


In [ ]:
nb_inputs = 100
nb_hidden = 4
nb_outputs = 2

In [ ]:
args = {'thres': -50,
        'u_rest': -60,
        'tau_mem': 1e-2,
        'tau_syn': 5e-3,
        'tau_ref': 5e-3,
        't_rise': 5e-3, # the pre-synaptic double exponential kernel rise time
        't_decay': 1e-2, # the pre-synaptic double exponential kernel decay time
        'timestep_size': 1e-4,
        't_rise_alpha': 5e-3,
        't_decay_alpha': 1e-2} 
